In [57]:
from collections import Counter
import operator

import pybaseball 

from pybaseball.lahman import batting, pitching, people, fielding

from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score

from pygam import LogisticGAM

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

### the run expectancy matrix

In [2]:
# function that converts base/out states to run expectancies using Tom Tango's "run expectancy matrix" (2010-15 data)
def get_run_expectancy(row, post=False):
   
    on_1b = 'on_1b'
    on_2b = 'on_2b'
    on_3b = 'on_3b'
    outs = 'outs_when_up'
       
    if not row[on_1b] and not row[on_2b] and not row[on_3b]:
        if row[outs] == 0:
            return 0.481
        elif row[outs] == 1:
            return 0.254
        elif row[outs] == 2:
            return 0.098
        else:
            return 0.
       
    if row[on_1b] and not row[on_2b] and not row[on_3b]:
        if row[outs] == 0:
            return 0.859
        elif row[outs] == 1:
            return 0.509
        elif row[outs] == 2:
            return 0.224
        else:
            return 0.
       
    if not row[on_1b] and row[on_2b] and not row[on_3b]:
        if row[outs] == 0:
            return 1.100
        elif row[outs] == 1:
            return 0.664
        elif row[outs] == 2:
            return 0.319
        else:
            return 0.
       
    if row[on_1b] and row[on_2b] and not row[on_3b]:
        if row[outs] == 0:
            return 1.437
        elif row[outs] == 1:
            return 0.884
        elif row[outs] == 2:
            return 0.429
        else:
            return 0.
       
    if not row[on_1b] and not row[on_2b] and row[on_3b]:
        if row[outs] == 0:
            return 1.350
        elif row[outs] == 1:
            return 0.950
        elif row[outs] == 2:
            return 0.353
        else:
            return 0.
       
    if row[on_1b] and not row[on_2b] and row[on_3b]:
        if row[outs] == 0:
            return 1.784
        elif row[outs] == 1:
            return 1.130
        elif row[outs] == 2:
            return 0.478
        else:
            return 0.
       
    if not row[on_1b] and row[on_2b] and row[on_3b]:
        if row[outs] == 0:
            return 1.964
        elif row[outs] == 1:
            return 1.376
        elif row[outs] == 2:
            return 0.580
        else:
            return 0.
       
    if row[on_1b] and row[on_2b] and row[on_3b]:
        if row[outs] == 0:
            return 2.292
        elif row[outs] == 1:
            return 1.541
        elif row[outs] == 2:
            return 0.752
        else:
            return 0.

### get event outcomes

In [3]:
# these are good end outcomes
valid_outcome_types = ['walk', 'hit_by_pitch', 'single', 'double', 'triple', 'home_run', 
                       'strikeout', 'field_error', 'sac_bunt', 'sac_fly']

# these are obviously ground_out-type of outcomes
ground_out_types = ['force_out', 'fielders_choice', 'fielders_choice_out']

# these are obivouly fly_out-type of outcomes
fly_out_types = ['sac_fly']

# general types of outcomes that need more investigation
general_out_types = ['field_out', 'double_play']

# function to get the outcome of each pitch
def get_outcome(event, des, description):
    if event in valid_outcome_types:
        return event
    elif event in ground_out_types:
        return 'ground_out'
    elif event in fly_out_types:
        return 'fly_out'
    elif event in general_out_types:
        if 'grounds' in des:
            return 'ground_out'
        elif 'flies' in des:
            return 'fly_out'
        elif 'pops' in des:
            return 'pop_out'
        elif 'lines' in des:
            return 'line_out'
    else:
        return event

### import pitch-by-pitch data

In [4]:
pitch_data = pd.read_csv("../data/pitch_data_2019.csv")

pitch_data = pitch_data[pitch_data['fielder_2.1'] == pitch_data['fielder_2.1']]

pitch_data = pitch_data.sort_values(by=['game_date', 'game_pk', 'inning', 'at_bat_number', 'pitch_number', 'outs_when_up'], ascending=True)

print(pitch_data.columns.tolist())

print(pitch_data.shape)



pitch_data.head()

['index', 'pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
731783,15276,FF,2019-03-28,96.0,-2.9039,5.2153,Luis Castillo,624428.0,622491.0,field_out,hit_into_play,NaN,NaN,NaN,NaN,7.0,"Adam Frazier grounds out, first baseman Joey V...",R,L,R,CIN,PIT,X,3.0,ground_ball,0.0,0.0,2019.0,-0.8590,0.8596,-0.3394,1.8839,NaN,NaN,NaN,0.0,1.0,Top,152.35,165.76,NaN,NaN,571466.0,NaN,190328_201138,8.4903,-139.4249,-5.0628,-12.5679,26.5388,-20.4739,3.301,1.9909,5.0,89.7,-23.2,94.839,2013.0,5.381,565220.0,622491.0,571466.0,458015.0,606299.0,553993.0,578428.0,608385.0,594988.0,624577.0,55.1186,0.116,0.112,0.0,1.0,0.0,0.0,2.0,1.0,1.0,4-Seam Fastball,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Standard,Standard
731782,15263,FF,2019-03-28,95.8,-2.7622,5.1903,Luis Castillo,466320.0,622491.0,NaN,called_strike,NaN,NaN,NaN,NaN,12.0,NaN,R,L,R,CIN,PIT,S,NaN,NaN,0.0,0.0,2019.0,-1.0101,0.9756,0.8866,2.7563,NaN,NaN,NaN,1.0,1.0,Top,NaN,NaN,NaN,NaN,571466.0,NaN,190328_201218,11.7184,-138.9595,-2.9584,-15.2994,28.0977,-19.3068,3.316,1.5126,NaN,NaN,NaN,95.034,2130.0,5.804,565220.0,622491.0,571466.0,458015.0,606299.0,553993.0,578428.0,608385.0,594988.0,624577.0,54.6959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,4-Seam Fastball,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard
731781,15244,CH,2019-03-28,88.6,-2.8765,5.0955,Luis Castillo,466320.0,622491.0,NaN,swinging_strike,NaN,NaN,NaN,NaN,14.0,NaN,R,L,R,CIN,PIT,S,NaN,NaN,0.0,1.0,2019.0,-1.1744,0.0629,0.1219,0.4152,NaN,NaN,NaN,1.0,1.0,Top,NaN,NaN,NaN,NaN,571466.0,NaN,190328_201232,9.4729,-128.5853,-5.4328,-14.3972,23.2579,-30.7459,3.300,1.5000,NaN,NaN,NaN,87.273,2240.0,5.314,565220.0,622491.0,571466.0,458015.0,606299.0,553993.0,578428.0,608385.0,594988.0,624577.0,55.1857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,Changeup,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard
731780,15227,FF,2019-03-28,96.4,-2.7735,5.2440,Luis Castillo,466320.0,622491.0,NaN,foul,NaN,NaN,NaN,NaN,1.0,NaN,R,L,R,CIN,PIT,S,NaN,NaN,0.0,2.0,2019.0,-1.0959,0.9177,-0.7734,3.4057,NaN,NaN,NaN,1.0,1.0,Top,NaN,NaN,NaN,NaN,571466.0,NaN,190328_201256,7.4728,-140.0436,-1.2863,-15.3898,29.5305,-20.6023,3.301,1.5040,257.0,82.3,44.1,94.236,2135.0,4.966,565220.0,622491.0,571466.0,458015.0,606299.0,553993.0,578428.0,608385.0,594988.0,624577.0,55.5331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,4-Seam Fastball,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard
731779,15212,CH,2019-03-28,88.0,-2.8687,4.9348,Luis Castillo,466320.0,622491.0,strikeout,swinging_strike,NaN,NaN,NaN,NaN,13.0,Melky Cabrera strikes out swinging.,R,L,R,CIN,PIT,S,2.0,NaN,0.0,2.0,2019.0,-1.1851,0.2630,-0.3882,1.1706,NaN,NaN,NaN,1.0,1.0,Top,NaN,NaN,NaN,NaN,571466.0,NaN,190328_201311,8.2811,-127.7756,-3.5437,-14.2871,23.6924,-28.9141,3.300,1.5000,NaN,NaN,NaN,87.118,2248.0,5.628,565220.0,622491.0,571466.0,458015.0,606299.0,553993.0,578428.0,608385.0,594988.0,624577.0,54.8713,NaN,NaN,0.0,1.0,0.0,0.0,NaN,2.0,4.0,Changeup,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard


## expected runs

To compute the expected runs:

- compute the run expectancy for each pitch in the data
- for each event, we compute the difference of the run expectancies before and after the event (this is what we call $\rho$)
- we then add to $\rho$ the difference between the batting team's score before and after the event (i.e., the "actual" number of runs that scored as a result of the event).
- this final quantity is what we call $\delta$ (or, more precisely, $\delta_i$ for the $i^{\rm th}$ event.

In [5]:
# make sure the pitch sequence is in order
exp_runs_df = pitch_data.copy()

# make "integer" type columns ints
for col in ['game_pk', 'inning', 'outs_when_up', 'bat_score']:
    exp_runs_df[col] = exp_runs_df[col].astype(int)
    
# convert on base columns to booleans
for col in ['on_1b', 'on_2b', 'on_3b']:
    exp_runs_df[col] = exp_runs_df[col].apply(lambda x: x == x)
    
exp_runs_df['runs_expected'] = exp_runs_df.apply(get_run_expectancy, axis=1)

# make a new id based on game id + pitcher id that we can use for groupby's
exp_runs_df['game_inning_id'] = exp_runs_df['game_pk'].astype(str) + '_' + exp_runs_df['inning'].astype(str) + '_' + exp_runs_df['inning_topbot']

# keep only pitches that end in an "event" (hit, out, etc.)
exp_runs_df = exp_runs_df[exp_runs_df['events'] == exp_runs_df['events']]

# keep only the columns that we need
exp_runs_df = exp_runs_df[['game_inning_id', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up', 'events', 'runs_expected', 'bat_score']]

# get previous game state
for col in ['runs_expected', 'bat_score']:
    exp_runs_df['following_'+col] = exp_runs_df.groupby('game_inning_id')[col].apply(lambda x: x.shift(-1))
    
# end of the inning plays result in the 25th base/out state which has expected runs of 0
exp_runs_df['following_runs_expected'] = exp_runs_df['following_runs_expected'].fillna(value=0)

# fill NaN's in the following_bat_score with the previous row's value
exp_runs_df['following_bat_score'].fillna(method='ffill', inplace=True)

# difference between final and initial run expectancies
exp_runs_df['rho'] = exp_runs_df['following_runs_expected'] - exp_runs_df['runs_expected']

# rho + actual runs scored
exp_runs_df['delta'] = exp_runs_df['rho'] + (exp_runs_df['following_bat_score'] - exp_runs_df['bat_score'])
print(exp_runs_df.shape)

exp_runs_df = exp_runs_df[['delta']]

exp_runs_df.head(5)

(185334, 12)


,delta
731783,-0.227
731779,-0.156
731774,0.126
731771,-0.224
731770,-0.227


# openWAR Model

## Apportioning Offensive Run Values

### Park effects and platoon advantage effects

We start with adjusting for park effects and platoon advantage effects (e.g., a left-handed batter against a right-handed pitcher).  We control for these factors by fitting a linear regression model to the offensive run values:

\begin{equation}
\delta_i = {\bf B}_i \cdot \alpha + \epsilon_i
\end{equation}

where the covariate vector ${\bf B}_i$ contains a set of indicator variables for the specific ballpark for plate
appearance $i$ and an indicator variable for whether or not the batter has a platoon advantage over
the pitcher. The coefficient vector ${\bf \alpha}$ contains the effects of each ballpark and the effect of a platoon
advantage on the offensive run values.

The residuals from the regression model:

\begin{equation}
\epsilon_i = \delta_i - {\bf B}_i \cdot \alpha
\end{equation}

represent the portion of the offensive run value $\delta_i$ that is not attributable to the ballpark or platoon
advantage, and so we refer to them as adjusted offensive run values.

In [6]:
# keep only the columns that we need
adj_off_runs_df = pitch_data[['home_team', 'p_throws', 'stand']]

# platoon-effect feature
def platoon_effect(row):
    
    if row['p_throws'] != row['stand']:
        return True
    else:
        return False
adj_off_runs_df['platoon_effect'] = adj_off_runs_df.apply(platoon_effect, axis=1)

# turn the home-team column into one-hot encoded features
temp_df = pd.get_dummies(adj_off_runs_df['home_team'])
adj_off_runs_df = pd.merge(adj_off_runs_df, temp_df, how='inner', left_index=True, right_index=True)
adj_off_runs_df.drop(['home_team', 'p_throws', 'stand'], axis=1, inplace=True)

# join with expected runs data
exp_runs_df = pd.merge(exp_runs_df, adj_off_runs_df, how='left', left_index=True, right_index=True)
exp_runs_df.head()

# train a linear regression model to adjust for park and platoon factors
X = exp_runs_df.drop('delta', axis=1)
y = exp_runs_df['delta']

linreg = LinearRegression(fit_intercept=False)

linreg.fit(X, y)

# extract the coefficient vector which contains the effects from each ballpark as well as the effects from platoon advantage
alphas = np.array(linreg.coef_)

# dot the coefficients back into the original matrix to compute the ballpark/platoon effect contributions to delta
B_dot_alpha = pd.DataFrame(np.matmul(X.as_matrix(), alphas))
B_dot_alpha.index = exp_runs_df.index

# merge with the original run expectancies
park_platoon_df = pd.merge(exp_runs_df[['delta']], B_dot_alpha, how='inner', left_index=True, right_index=True)
park_platoon_df.columns = ['delta', 'B_dot_alpha']

# compute the residuals (the part of the run values that are NOT attributable to the ballpark and/or platoons)
park_platoon_df['epsilon'] = park_platoon_df['delta'] - park_platoon_df['B_dot_alpha']

park_platoon_df = park_platoon_df[['epsilon']]

park_platoon_df.head()

,epsilon
731783,-0.237652
731779,-0.166652
731774,0.115348
731771,-0.234652
731770,-0.237652


### Baserunner run values

The next task is determining the portion of ˆi that is attributable to the baserunners for each plate
appearance i based on the following principle: baserunners should only get credit for advancement
beyond what would be expected given their starting locations, the number of outs, and the hitting
event that occurred. We can estimate this expected baserunner advancement by fitting a second
regression model to our adjusted offensive run values:

\begin{equation}
\epsilon = {\bf S}_i \cdot {\bf \beta} + \eta_i
\end{equation}

where the covariate vector ${\bf S}_i$ consists of: 1) a set of indicator variables that indicate the specific
game state (number of outs, locations of baserunners) at the start of plate appearance i and; 2)
the hitting event (e.g. single, double, etc.) that occurred during plate appearance i.

The residuals from the regression model:

\begin{equation}
\eta = \epsilon - {\bf S}_i \cdot {\bf \beta}
\end{equation}

represent the portion of the adjusted offensive run value that is attributable to the baserunners.
If the baserunners take extra bases beyond what is expected, then $\eta_i$ will be positive, whereas if
they take fewer bases or get thrown out then $\eta_i$ will be negative. Note that $\eta_i$ also contains the
baserunning contribution of the hitter for plate appearance $i$.

In [7]:
# keep only the columns that we need
baserunner_runs_df = pitch_data[['on_1b', 'on_2b', 'on_3b', 'outs_when_up', 'events', 'des', 'description']]
baserunner_runs_df = baserunner_runs_df[baserunner_runs_df['events'] == baserunner_runs_df['events']]

baserunner_runs_df['pitch_outcome'] = baserunner_runs_df.apply(lambda x: get_outcome(x['events'], x['des'], x['description']), axis=1)

event_count_dict = dict(Counter(baserunner_runs_df['pitch_outcome']))
sorted_event_count_dict = sorted(event_count_dict.items(), key=operator.itemgetter(1), reverse=True)

# print("=============================")
# print("Types of events and frequency")
# print("=============================")
# for k in sorted_event_count_dict:
#     print(f"{k[0]}: {round(k[1] / len(baserunner_runs_df), 3)}")

# convert on base columns to booleans
for col in ['on_1b', 'on_2b', 'on_3b']:
    baserunner_runs_df[col] = baserunner_runs_df[col].apply(lambda x: x == x)
    
# turn the home-team column into one-hot encoded features
temp_df = pd.get_dummies(baserunner_runs_df['pitch_outcome'])
baserunner_runs_df = pd.merge(baserunner_runs_df, temp_df, how='inner', left_index=True, right_index=True)
baserunner_runs_df.drop(['pitch_outcome', 'des', 'description', 'events'], axis=1, inplace=True)

# join with adjusted expected runs data
adj_exp_runs_df = pd.merge(park_platoon_df, baserunner_runs_df, how='left', left_index=True, right_index=True)
adj_exp_runs_df.head()

# train a linear regression model to extract portion of run values attributable to runners
X = adj_exp_runs_df.drop('epsilon', axis=1)
y = adj_exp_runs_df['epsilon']

linreg = LinearRegression(fit_intercept=False)
linreg.fit(X, y)

# extract the coefficient vector 
betas = np.array(linreg.coef_)

# dot the coefficients back into the original matrix to compute the ballpark/platoon effect contributions to delta
S_dot_beta = pd.DataFrame(np.matmul(X.as_matrix(), betas))
S_dot_beta.index = adj_exp_runs_df.index

# merge with the original run expectancies
baserunner_df = pd.merge(adj_exp_runs_df[['epsilon']], S_dot_beta, how='inner', left_index=True, right_index=True)
baserunner_df.columns = ['epsilon', 'S_dot_beta']

# compute the residuals (the part of the run values that are NOT attributable to the ballpark and/or platoons)
baserunner_df['eta'] = baserunner_df['epsilon'] - baserunner_df['S_dot_beta']

baserunner_df = baserunner_df[['eta']]

baserunner_df.head()

,eta
731783,-0.00902052
731779,0.109223
731774,-0.174922
731771,-0.000454691
731770,0.0366402


### Calculating Runs Above Average for Baserunners

In [38]:
# keep only the columns that we need
baserunner_raa_df = pitch_data[['game_pk', 'inning', 'inning_topbot', 'outs_when_up', 'batter', 'on_1b', 'on_2b', 'on_3b', 'events', 'des', 'description']]

for col in ['game_pk', 'inning', 'batter']:
    baserunner_raa_df[col] = baserunner_raa_df[col].astype(int)    

for col in ['on_1b', 'on_2b', 'on_3b']:
    baserunner_raa_df[col] = baserunner_raa_df[col].fillna(value=0.).astype(int)

# make a new id based on game id + pitcher id that we can use for groupby's
baserunner_raa_df['game_inning_id'] = baserunner_raa_df['game_pk'].astype(str) + '_' + baserunner_raa_df['inning'].astype(str) + '_' + baserunner_raa_df['inning_topbot']

# get previous game state
for col in ['on_1b', 'on_2b', 'on_3b']:
    baserunner_raa_df['following_'+col] = baserunner_raa_df.groupby('game_inning_id')[col].apply(lambda x: x.shift(-1)).fillna(value=0.).astype(int)
    
# for col in ['following_on_1b', 'following_on_2b', 'following_on_3b']:
#     baserunner_raa_df[col] = baserunner_raa_df[col].astype(int)

baserunner_raa_df = baserunner_raa_df[baserunner_raa_df['events'] == baserunner_raa_df['events']]

baserunner_raa_df['pitch_outcome'] = baserunner_raa_df.apply(lambda x: get_outcome(x['events'], x['des'], x['description']), axis=1)

baserunner_raa_df = baserunner_raa_df[['pitch_outcome', 'batter', 'on_1b', 'on_2b', 'on_3b', 'following_on_1b', 'following_on_2b', 'following_on_3b']]

baserunner_raa_df.head(10)

,pitch_outcome,batter,on_1b,on_2b,on_3b,following_on_1b,following_on_2b,following_on_3b
731783,ground_out,624428,0,0,0,0,0,0
731779,strikeout,466320,0,0,0,0,0,0
731774,walk,572816,0,0,0,572816,0,0
731771,strikeout,605137,572816,0,0,0,0,0
731770,line_out,608385,0,0,0,0,0,0
731766,double,458015,0,0,0,0,458015,0
731759,strikeout,624577,0,458015,0,0,458015,0
731756,pop_out,553993,0,458015,0,0,0,0
731752,fly_out,465041,0,0,0,0,0,0
731747,strikeout,628356,0,0,0,0,0,0


In [40]:
baserunner_raa_df['pitch_outcome'].unique()

array(['ground_out', 'strikeout', 'walk', 'line_out', 'double', 'pop_out',
       'fly_out', 'single', 'sac_bunt', 'field_error', 'home_run',
       'triple', 'sac_fly', 'hit_by_pitch', 'grounded_into_double_play',
       'caught_stealing_2b', 'pickoff_caught_stealing_2b', 'pickoff_1b',
       'strikeout_double_play', 'run', 'sac_bunt_double_play',
       'caught_stealing_home', 'other_out', 'catcher_interf',
       'caught_stealing_3b', None, 'sac_fly_double_play',
       'batter_interference', 'pickoff_2b',
       'pickoff_caught_stealing_home', 'triple_play',
       'pickoff_caught_stealing_3b', 'pickoff_3b'], dtype=object)

### Hitter run values

The remaining portion of the adjusted offensive run value (after the contribution from baserunners is subtracted) is attributed to the hitter:

\begin{equation}
\mu_i = \epsilon_i - \eta_i
\end{equation}

Our remaining task for hitters is to calibrate their hitting performance relative to the expected hitting performance
based on all players at the same fielding position. We fit another linear regression model to adjust the hitter run value by the hitter’s fielding position:

\begin{equation}
\mu_i = {\bf H}_i \cdot {\bf \gamma} + \nu_i
\end{equation}

where the covariate vector ${\bf H}_i$ consists of a set of indicator variables for the fielding position of the
hitter in plate appearance $i$.  Note that pinch-hitter (PH) and designated hitter (DH) are also valid
values for hitter position. 

Estimated coefficients γb are calculated by ordinary least squares using
every plate appearance in our dataset. The estimated residuals from this regression model,

\begin{equation}
{\rm RAA}_i^{hit} = \nu = \mu_i - {\bf H}_i \cdot {\bf \gamma}
\end{equation}

represent the run values (above the average for the hitter’s position) for the hitter in each plate
appearance $i$.

In [8]:
hitter_df = pd.merge(park_platoon_df, baserunner_df, how='inner', left_index=True, right_index=True)

# first, compute mu
hitter_df['mu'] = hitter_df['epsilon'] - hitter_df['eta']
hitter_df.drop(['epsilon', 'eta'], axis=1, inplace=True)

# function to find the batter's fielding position
def get_fielding_position(row):

    temp_df = pitch_data[pitch_data['game_pk'] == row['game_pk']]
    temp_df = temp_df[['game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9']]
    temp_df = temp_df.astype(int)
    
    pos_dict = {
        'pitcher': list(temp_df['pitcher.1'].unique()),
        'catcher': list(temp_df['fielder_2.1'].unique()),
        'firstbase': list(temp_df['fielder_3'].unique()),
        'secondbase': list(temp_df['fielder_4'].unique()),
        'thirdbase': list(temp_df['fielder_5'].unique()),
        'shortstop': list(temp_df['fielder_6'].unique()),
        'leftfield': list(temp_df['fielder_7'].unique()),
        'centerfield': list(temp_df['fielder_8'].unique()),
        'rightfield': list(temp_df['fielder_9'].unique())
    }
        
    for k,v in pos_dict.items():
        if row['batter'] in v:
            return k
    
# dataframe with just the game ID and batter ID    
pos_df = pitch_data[['game_pk', 'batter']]
pos_df = pos_df.astype(int)

# merge with the mu dataframe
pos_df = pd.merge(hitter_df, pos_df, how='left', left_index=True, right_index=True)

# get the batter's fielding position
pos_df['batter_fld_pos'] = pos_df.apply(get_fielding_position, axis=1)
pos_df['batter_fld_pos'] = pos_df['batter_fld_pos'].fillna(value='dh-ph')

# one-hot encode fielding positions
temp_df = pd.get_dummies(pos_df['batter_fld_pos'])
fld_pos_df = pd.merge(hitter_df, temp_df, how='left', left_index=True, right_index=True)

# train a linear regression model to extract portion of run values attributable to runners
X = fld_pos_df.drop('mu', axis=1)
y = fld_pos_df['mu']

linreg = LinearRegression(fit_intercept=False)
linreg.fit(X, y)

# extract the coefficient vector 
gammas = np.array(linreg.coef_)

# dot the coefficients back into the original matrix to compute the ballpark/platoon effect contributions to delta
H_dot_gamma = pd.DataFrame(np.matmul(X.as_matrix(), gammas))
H_dot_gamma.index = fld_pos_df.index

# merge with the original run expectancies
hitters_df = pd.merge(fld_pos_df[['mu']], H_dot_gamma, how='inner', left_index=True, right_index=True)
hitters_df.columns = ['mu', 'H_dot_gamma']

# compute the residuals (the part of the run values that are NOT attributable to the ballpark and/or platoons)
hitters_df['nu'] = hitters_df['mu'] - hitters_df['H_dot_gamma']

hitters_df = hitters_df[['nu']]

hitters_df.head()

,nu
731783,-0.226475
731779,-0.289777
731774,0.279107
731771,-0.247301
731770,-0.285456


### Calculating Runs Above Average for Hitting

In [16]:
RAA_hitting_df = pd.merge(hitters_df, pitch_data[['batter']], how='left', left_index=True, right_index=True)
RAA_hitting_df['batter'] = RAA_hitting_df['batter'].astype(int)

RAA_hitting_df = pd.DataFrame(RAA_hitting_df.groupby('batter')['nu'].sum())
RAA_hitting_df.reset_index(inplace=True, drop=False)

RAA_hitting_df.columns = ['player_id', 'RAA_hitting']

RAA_hitting_df.sort_values(by='RAA_hitting', inplace=True, ascending=False)

RAA_hitting_df.head(10)

,player_id,RAA_hitting
295,545361,62.023172
439,592885,51.522081
605,608324,46.823042
577,606466,46.394649
753,641355,45.442134
276,543685,41.270797
31,443558,39.715551
444,593428,38.058747
982,670541,34.088458
281,543807,33.252648


## Apportioning Defensive Run Values

Now, we must apportion the defensive run value $−\delta_{i}$ between the pitcher and
various fielders involved in plate appearance $i$.

The degree to which the pitcher (versus the fielders) is responsible for the run value of a ball
in play depends on how difficult that batted ball was to field. Surely, if the pitcher allows a batter
to hit a home run, the fielders share no responsibility for that run value. Conversely, if a routine
groundball is muffed by the fielder, the pitcher should bear very little responsibility.

We assign the entire defensive run value $−\delta_i$ to the pitcher for any plate appearance that does
not result in a ball in play (e.g. strikeout, home run, hit by pitch, etc.). For balls hit into play, we
must estimate the probability p that each ball-in-play would result in an out given the location that
ball in play was hit.

The MLBAM data set contains $(x, y)$-coordinates that give the location of each batted ball, and
we use a two-dimensional kernel density smoother to estimate the probability of an
out at each coordinate in the field:

\begin{equation}
p_i = f(x, y)
\end{equation}

To construct this probability function we will fit a "generalized additive model" using the package pygam.

For that ball in play $i$, we use $p_i$ to divide the responsibility
between the pitcher and the fielders. Specifically, we apportion:

\begin{eqnarray}
\delta_i^p &=& - \delta_i \, (1 - p_i) \,\,\,\,\, {\rm (for \, the \, pitcher)} \\
\delta_i^f &=& - \delta_i \, p_i \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\, {\rm (for \, the \, fielder)}
\end{eqnarray}

The fielders bear more responsibility for a ball in play that is relatively easy to field ($p_i$ near 1)
whereas a pitcher bears more responsibility for a ball in play that is relatively hard to field ($p_i$ near
0).

### The Out Probability Model

In [63]:
out_prob_df = pitch_data[['events', 'hc_x', 'hc_y']]
out_prob_df.dropna(inplace=True)

out_prob_df = out_prob_df[out_prob_df['events'] == out_prob_df['events']]

# keep only balls in play
bip_outcomes = ['field_out', 'double', 'single', 'sac_bunt', 'field_error',
                'force_out', 'triple', 'sac_fly', 
                'fielders_choice_out', 'grounded_into_double_play', 'double_play',
                'fielders_choice', 'sac_bunt_double_play',
                'sac_fly_double_play', 'triple_play']
out_prob_df = out_prob_df[out_prob_df['events'].isin(bip_outcomes)]

# convert events to boolean (whether an out was recorded or not)
def bip_out(x):
    if x not in ['single', 'double', 'triple', 'field_error']:
        return True
    else:
        return False
out_prob_df['events'] = out_prob_df['events'].apply(bip_out)

# fit a generalized additive model
X = out_prob_df[['hc_x', 'hc_y']]
y = out_prob_df[['events']]

gam = LogisticGAM().fit(X, y)

print(f"Accuracy of predictions: {accuracy_score(y, gam.predict(X))}")
print(f"ROC AUC: {roc_auc_score(y, gam.predict_proba(X))}")

Accuracy of predictions: 0.7532209522521304
ROC AUC: 0.7895453229222938


In [68]:
pitch_data['events'].unique()

array(['field_out', nan, 'strikeout', 'walk', 'double', 'single',
       'sac_bunt', 'field_error', 'force_out', 'home_run', 'triple',
       'sac_fly', 'hit_by_pitch', 'fielders_choice_out',
       'grounded_into_double_play', 'double_play', 'caught_stealing_2b',
       'pickoff_caught_stealing_2b', 'pickoff_1b',
       'strikeout_double_play', 'fielders_choice', 'run',
       'sac_bunt_double_play', 'caught_stealing_home', 'other_out',
       'catcher_interf', 'caught_stealing_3b', 'sac_fly_double_play',
       'batter_interference', 'pickoff_2b',
       'pickoff_caught_stealing_home', 'triple_play',
       'pickoff_caught_stealing_3b', 'pickoff_3b'], dtype=object)

### Allocating Defensive Runs

In [100]:
defensive_runs_df = pitch_data[['events', 'hc_x', 'hc_y']]
defensive_runs_df = pd.merge(-exp_runs_df[['delta']], defensive_runs_df, how='left', left_index=True, right_index=True)

def_out_prob_df = defensive_runs_df[(defensive_runs_df['hc_x'] == defensive_runs_df['hc_x']) & (defensive_runs_df['hc_y'] == defensive_runs_df['hc_y'])]
def_out_prob_df = def_out_prob_df[['hc_x', 'hc_y']]
def_out_prob_df['out_probability'] = gam.predict_proba(def_out_prob_df)
def_out_prob_df = def_out_prob_df[['out_probability']]

defensive_runs_df = pd.merge(defensive_runs_df, def_out_prob_df, how='left', left_index=True, right_index=True)

def get_pitching_runs(row):
    if row['events'] in ['strikeout', 'walk', 'home_run', 'hit_by_pitch']:
        return row['delta']
    else:
        return 0
defensive_runs_df['pitching_runs'] = defensive_runs_df.apply(get_pitching_runs, axis=1)
defensive_runs_df['pitcher_fld_runs'] = -defensive_runs_df['delta'] * (1 - defensive_runs_df['out_probability'])
defensive_runs_df['player_defense_runs'] = -defensive_runs_df['delta'] * defensive_runs_df['out_probability']
defensive_runs_df.fillna(value=0., inplace=True)

defensive_runs_df['pitcher_defense_runs'] = defensive_runs_df['pitching_runs'] + defensive_runs_df['pitcher_fld_runs']

defensive_runs_df = defensive_runs_df[['pitcher_defense_runs', 'player_defense_runs']]

defensive_runs_df.head()

,pitcher_defense_runs,player_defense_runs
731783,-0.016056,-0.210944
731779,0.156000,0.000000
731774,-0.126000,0.000000
731771,0.224000,0.000000
731770,-0.141509,-0.085491


### import batters

### import pitchers